In [10]:
import io
import requests
import pandas as pd
from datetime import datetime, timezone

In [14]:

def fetch_data():
    timestamp = int(datetime.now(timezone.utc).timestamp())
    data = {"timestamp": timestamp}
    
    r = requests.get("https://mempool.space/api/v1/fees/mempool-blocks")
    if r.ok:
        block_data = r.json()[0]
        block_data.pop("feeRange", None)
        data.update({f"mempool_blocks_{k}": v for k, v in block_data.items()})
    
    r = requests.get("https://mempool.space/api/v1/fees/recommended")
    if r.ok:
        data.update({f"recommended_fee_{k}": v for k, v in r.json().items()})
    
    r = requests.get("https://mempool.space/api/mempool")
    if r.ok:
        data.update({f"mempool_{k}": v for k, v in r.json().items()})
    
    r = requests.get("https://mempool.space/api/v1/difficulty-adjustment")
    if r.ok:
        data.update({f"difficulty_adjustment_{k}": v for k, v in r.json().items()})
    
    r = requests.get("https://mempool.space/api/v1/prices")
    if r.ok:
        data.update({f"price_{k}": v for k, v in r.json().items()})
    
    return flatten_data(data)

def flatten_data(data):
    fee_hist_key = "mempool_fee_histogram"
    if fee_hist_key in data and isinstance(data[fee_hist_key], list):
        histogram = data.pop(fee_hist_key)
        bins = []
        for lower in range(1, 10):
            bins.append((lower, lower+1, f"{lower}_{lower+1}"))
        for lower in range(10, 30, 2):
            bins.append((lower, lower+2, f"{lower}_{lower+2}"))
        for lower in range(30, 100, 5):
            bins.append((lower, lower+5, f"{lower}_{lower+5}"))
        for lower in range(100, 1000, 50):
            bins.append((lower, lower+50, f"{lower}_{lower+50}"))
        bins.append((1000, float('inf'), "1000_plus"))
        
        bin_counts = {f"mempool_fee_histogram_bin_{label}": 0 for _, _, label in bins}
        for pair in histogram:
            if isinstance(pair, list) and len(pair) == 2:
                fee, count = pair
                for lower, upper, label in bins:
                    if lower <= fee < upper:
                        bin_counts[f"mempool_fee_histogram_bin_{label}"] += count
                        break
        data.update(bin_counts)
    
    return data

if __name__ == "__main__":
    record = fetch_data()
    df = pd.DataFrame([record]).astype("float64")

    drop_cols = [
        "difficulty_adjustment_estimatedRetargetDate",
        "difficulty_adjustment_previousRetarget",
        "price_time"
    ]
    df = df.drop(columns=drop_cols, errors="ignore")

df['timestamp'] = df['timestamp'].astype('int64')

In [15]:
df

,timestamp,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,...,mempool_fee_histogram_bin_550_600,mempool_fee_histogram_bin_600_650,mempool_fee_histogram_bin_650_700,mempool_fee_histogram_bin_700_750,mempool_fee_histogram_bin_750_800,mempool_fee_histogram_bin_800_850,mempool_fee_histogram_bin_850_900,mempool_fee_histogram_bin_900_950,mempool_fee_histogram_bin_950_1000,mempool_fee_histogram_bin_1000_plus
0,1747206001,1572645.0,997995.75,3703.0,9026363.0,4.487442,5.0,4.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df["recommended_fee_fastestFee"]

0    4.0
Name: recommended_fee_fastestFee, dtype: float64